In [33]:
import json
import pandas as pd
import requests
from tqdm import tqdm

In [34]:
# STEP 0 - setting Environment Variables

country = "fr"
api_id = "37fe08af"
api_key = "17c1369ea8fb68bc48f834aebc0bec53"
results_per_page = 25
page_scraped = 100
what_and = "data%20engineer"
max_days_old = str(30)

In [35]:
# ETAPE 1 connexion à l'API + récupération offres

def create_url(start_url="http://api.adzuna.com/v1/api/jobs/"):
    """
    concactène l'adresse url à partir des paramètres choisis
    comment améliorer pour que les paramètres soient correctement passés ?
    """
    url_list = []
    for page_number in range(1, page_scraped):
        #start_url = "http://api.adzuna.com/v1/api/jobs/"
        #country = "fr"
        part0_url = "/search/"
        number_page = str(page_number) 
        part1_url = "?app_id="
        #api_id = "37fe08af"
        part2_url = "&app_key="
        #api_key = "17c1369ea8fb68bc48f834aebc0bec53"
        part3_url = "&results_per_page="
        offer_per_page = str(results_per_page)
        part4_url = "&what_and="
        #what_and = "data%20engineer"
        part5_url = "&max_days_old="
        #max_days_old = str(30)
        end_url = "&content-type=application/json"
        full_url = f"{start_url}{country}{part0_url}{number_page}{part1_url}{api_id}{part2_url}{api_key}{part3_url}{results_per_page}{part4_url}{what_and}{part5_url}{max_days_old}{end_url}"
        url_list.append(full_url)
    return url_list

In [36]:
def scrape_urls(url):
    """
    lance la requête sur l'api Adzuna pour chacunes des url crééés
    si le status_code est incorrect, il s'affiche et l'url liée s'affiche
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("The request for {} was not successful : status {}".format(url, response.status_code))
        return None

In [37]:
class Company:
    def __init__(self):
        self.name = None
        self.sector = None
        self.employees = None
        self.creation_year = None
        self.turnover = None
        self.mean_age = None


class JobOffer:
    def __init__(self):
        self.title = None
        self.company = None
        self.contract_type = None
        self.location = None
        self.salary = None
        self.remote_type = None
        self.starting_date = None
        self.required_experience = None
        self.education = None
        self.description = None
        self.profil_experience = None
        self.publication_date = None
        self.url_direct_offer = None
        

class GetJobOffer():
    def __init__(self, full_info):
        self.full_info = full_info
        self.job_offer = JobOffer()
        self.company = Company()
        

    def get_job_details(self):
        results = self.full_info
        self.job_offer.title = results["title"] if "title" in results else None 
        self.job_offer.location = results["location"]["display_name"] if "location" in results else None
        self.job_offer.company = vars(self.company)
        if ("contract_type" in results) or ("contract_time" in results):
            if ("contract_type" in results) and ("contract_time" in results):
                self.job_offer.contract_type = results["contract_type"] + " and " + results["contract_time"]
            elif ("contract_type" in results) and ("contract_time" not in results):
                self.job_offer.contract_type = results["contract_type"] 
            else : 
                self.job_offer.contract_type = results["contract_time"] 
        self.job_offer.salary = results["salary_is_predicted"] if "salary_is_predicted" in results else None
        self.job_offer.description = results["description"] if "description" in results else None
        self.job_offer.publication_date = results["created"][:10] if "created" in results else None
        self.job_offer.url_direct_offer = results["redirect_url"] if "redirect_url" in results else None
        
    
    def get_company_details(self):
        results = self.full_info
        if "category" in results:
            self.company.sector = results["category"]["tag"]
        if 'company' in results:
            if "display_name" in results['company']:
                self.company.name = results['company']["display_name"]
            elif "name" in results['company']:
                self.company.name = results['company']["name"]


    def combine_job_company(self):
        self.get_company_details()
        self.get_job_details()
        return vars(self.job_offer)


In [38]:
def final_step(jobs_offers):
    jobs_ready_for_export = []
    for offer in jobs_offers:
        nb_offers_on_page = 0
        while nb_offers_on_page < results_per_page:
            getjoboffer = GetJobOffer(offer["results"][nb_offers_on_page])
            jobs_ready_for_export.append(getjoboffer.combine_job_company())
            nb_offers_on_page += 1
    return jobs_ready_for_export    

In [39]:
url_list = create_url()

jobs_offers = [scrape_urls(url) for url in tqdm(url_list, desc="Accessing API")]

jobs_ready_for_export = final_step(jobs_offers)

df_adzuna_jobs = pd.DataFrame(jobs_ready_for_export)

df_adzuna_jobs.to_csv('output/job_offers_adzuna.csv')

df_adzuna_jobs.describe()

Accessing API: 100%|█| 99/99 [03:17<00:00,  1.99s/


,title,company,contract_type,location,salary,remote_type,starting_date,required_experience,education,description,profil_experience,publication_date,url_direct_offer
count,2475,2475,1105,2475,2475,0,0,0,0,2475,0,2475,2475
unique,1651,1218,6,279,1,0,0,0,0,1977,0,31,2475
top,Data Engineer H/F,"{'name': 'Amazon TA', 'sector': 'unknown', 'em...",permanent,"Paris, Ile-de-France",0,NaN,NaN,NaN,NaN,Amazon Hardlines Private Brands is looking for...,NaN,2024-02-23,https://www.adzuna.fr/land/ad/4574739084?se=1i...
freq,84,64,395,696,2475,NaN,NaN,NaN,NaN,64,NaN,197,1


La variable *jobs_offers* est un objet json donc type dictionnaire
on instancie *full_info* de la class GetJobOffer avec l'intégralité de *jobs_offers*

Les informations clés liés au poste se trouve dans *jobs_offers[results]* qui est une liste d'un seul élément, un dictionnaire dont chaque clé est une information utile 
- 'location' (elle même un dictionnaire)
    exemple 'location': {'__CLASS__': 'Adzuna::API::Response::Location', 'display_name': 'Toulouse, Haute-Garonne', 'area': ['France', 'Occitanie', 'Haute-Garonne', 'Toulouse']},
- 'category' (elle même un dictionnaire)
    exemple {'label': 'Unknown', '__CLASS__': 'Adzuna::API::Response::Category', 'tag': 'unknown'}
- title': 'Data Engineer PySpark'
- 'redirect_url': 'https://www.adzuna.fr/land/ad/4574739084?se=DGYosEfS7hGqg0-VTaQgFg&utm_medium=api&utm_source=37fe08af&v=FE7ECBBD26503585482CAD5270E4346D7E31A2D2'
- 'contract_time': 'full_time'
- 'contract_type': 'permanent'
- 'id': '4574739084'
- 'created': '2024-02-19T20:01:48Z'
- 'description'
    "QUI SOMMES-NOUS ? Thales propose des systèmes d'information et de communication sécurisés et interopérables pour les forces armées, les forces de sécurité et les opérateurs d'importance vitale. Ces activités, qui regroupent radiocommunications, réseaux, systèmes de protection, systèmes d'information critiques et cybersécurité, répondent aux besoins de marchés où l'utilisation des nouvelles technologies numériques est déterminante. Thales intervient tout au long de la chaîne de valeur, des équip…"
- 'salary_is_predicted': '0'
- 'company'(elle même un dictionnaire)
      {'display_name': 'THALES', '__CLASS__': 'Adzuna::API::Response::Company'}